In [71]:
import boto3
import json
import os
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from IPython.display import Audio

In [72]:
def clean_title(title):
    """출처 정보를 제거하는 함수"""
    # 출처 정보(예: "네이버 블로그", "출처:", "Facebook", "Instagram") 등을 제거하는 정규 표현식
    title = re.sub(r'\[.*\]|\s*(네이버\s*블로그|Tripadvisor|트립어드바이저|다이닝코드|빅데이터|맛집검색|Facebook|Instagram|위키피디아|출처:.*|출처\s*.*)', '', title)  # 출처 부분 제거
    return title.strip()

In [73]:
bedrock = boto3.client('bedrock-runtime', region_name='ap-northeast-2')

In [74]:
# model_id = 'anthropic.claude-3-5-sonnet-20240620-v1:0'
model_id = 'anthropic.claude-3-haiku-20240307-v1:0'

In [75]:
system_prompt = '''
[role]
- 당신은 주어진 장소의 정보를 읽고 그 장소를 여행하는 사람에게 정보를 정리하여 전달하는 아나운서입니다.
- 정보는 한국어로 번역하여 전달합니다.
- 장소의 정보는 여행하는 사람이 흥미있어할 만한 내용을 위주로 정리합니다.
- 흥미있는 내용이 있다면 꼭 포함해주세요.

[current_year]
2024

[behavior_guidelines]
- 정확한 정보만 제공하며, 문서에 없는 내용을 상상하여 답변하지 않습니다.
- '문서에에 따르면'과 같이 문서를 참고하는 것을 티내지 마세요.
- 사용자에게 바로 설명할 수 있도록 답변하세요.
- 아나운서의 대본처럼 바로 읽을 수 있게 답변하세요
'''

In [76]:
infer_system_prompt = '''
[role]
- 당신은 주어진 검색 결과를 보고 어떤 장소 혹은 물건에 대한 결과인지 유추합니다.
- 정보는 한국어로 번역하여 전달합니다.

[current_year]
2024

[behavior_guidelines]
- 유추한 물건 이름만 반환합니다.

[response example]
미륵사지석탑
'''

In [77]:
options = Options() 
options.add_argument("-headless")

In [78]:
driver = webdriver.Firefox()
# driver = webdriver.Firefox(options=options)

In [79]:
driver.implicitly_wait(10)

In [80]:
driver.get('https://images.google.com/')

In [81]:
driver.find_element(By.CSS_SELECTOR, "div[aria-label='이미지로 검색']").click()

In [82]:
drag_n_drop_area = driver.find_element(By.CSS_SELECTOR, "input[type='file']")

In [83]:
drag_n_drop_area.send_keys(os.path.abspath('./다운로드.jpeg'))

In [92]:
wiki = driver.find_element(By.CSS_SELECTOR, "div[data-attrid='description']")
wiki_url = wiki.find_element(By.CSS_SELECTOR, 'a[href]').get_attribute('href')

NoSuchElementException: Message: Unable to locate element: div[data-attrid='description']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16


In [84]:
google_search = driver.find_element(By.CSS_SELECTOR, "div.wNPKTe")
google_search.click()

NoSuchElementException: Message: Unable to locate element: div.wNPKTe; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16


In [85]:
# 반환값이 없을 때 수행할 코드
image_titles = []
title_elements = driver.find_elements(By.CSS_SELECTOR, "div.UAiK1e")  # 제목 요소 CSS 선택자 지정

# 상단 20개의 이미지 제목 수집
for i, title_element in enumerate(title_elements[:30]):
    title_text = title_element.text
    cleaned_text = clean_title(title_text)  # 불필요한 정보 제거
    image_titles.append(cleaned_text)

In [86]:
messages =  [{"role": "user", "content": [{"text":'\n'.join(image_titles)}]}]

In [87]:
response = bedrock.converse(
    modelId=model_id,
    system=[{"text":infer_system_prompt}],
    messages=messages,
    inferenceConfig={
        "maxTokens": 4096,   
    }
)

In [89]:
# google_search.find_element(By.XPATH, "..").find_element(By.CSS_SELECTOR, "h2").text

In [90]:
# driver.close()

In [91]:
# driver.switch_to.window(driver.window_handles[0])

In [32]:
document_area = driver.find_element(By.CSS_SELECTOR, "div[id='kp-wp-tab-overview']")

In [33]:
# documents = document_area.find_elements(By.CSS_SELECTOR, "h3.LC20lb")

In [34]:
# urls = [doc.find_element(By.XPATH, "..").get_attribute('href') for doc in documents]

In [35]:
# texts = list()
# for url in urls:
#     try:
#         texts.append(
#             BeautifulSoup(requests.get(url).text).get_text(strip=True)
#         )
#     except:
#         continue

In [36]:
# content = '\n\n'.join(texts)

In [44]:
wiki = driver.find_element(By.CSS_SELECTOR, "div[data-attrid='description']")

In [45]:
wiki_url = wiki.find_element(By.CSS_SELECTOR, 'a[href]').get_attribute('href')

In [46]:
response = requests.get(wiki_url)

In [47]:
soup = BeautifulSoup(response.text)

In [48]:
content = soup.find('div', {"id":"mw-content-text"}).text.strip()

In [49]:
messages =  [{"role": "user", "content": [{"text":content}]}]

In [66]:
system_prompt = '''
[role]
- 당신은 주어진 장소의 정보를 읽고 그 장소를 여행하는 사람에게 정보를 정리하여 전달하는 아나운서입니다.
- 정보는 Hindi로 번역하여 전달합니다.
- 장소의 정보는 여행하는 사람이 흥미있어할 만한 내용을 위주로 정리합니다.
- 흥미있는 내용이 있다면 꼭 포함해주세요.

[current_year]
2024

[behavior_guidelines]
- 정확한 정보만 제공하며, 문서에 없는 내용을 상상하여 답변하지 않습니다.
- '문서에에 따르면'과 같이 문서를 참고하는 것을 티내지 마세요.
- 사용자에게 바로 설명할 수 있도록 답변하세요.
- 아나운서의 대본처럼 바로 읽을 수 있게 답변하세요
'''

In [67]:
response = bedrock.converse(
    modelId=model_id,
    system=[{"text":system_prompt}],
    messages=messages,
    inferenceConfig={
        "maxTokens": 4096,   
    }
)

In [68]:
script = response['output']['message']['content'][0]['text']

In [69]:
script = re.sub('\n+', '\n', script)

In [70]:
print(script)

नमस्ते! मैं आपको सेंटरफील्ड (CENTERFIELD) के बारे में जानकारी प्रदान करता हूं।
सेंटरफील्ड दक्षिण कोरिया के सियोल शहर के गंगनाम-गु में स्थित एक बहुमंजिला इमारत है। यह परियोजना तेहेरान रोड 237 पुनर्निर्माण प्रोजेक्ट के हिस्से के रूप में है।
वेस्ट टावर 36 मंजिल का है और कार्यालय/होटल सुविधाएं हैं, जबकि ईस्ट टावर 35 मंजिल का है और केवल कार्यालय सुविधाएं हैं। खुला सार्वजनिक स्थान (3), सहायक प्रवेश द्वार (3), मुख्य प्रवेश द्वार (2) और वाहन निकास द्वार हैं। इस परियोजना का क्षेत्रफल 239,957.07 वर्ग मीटर है।
इसकी वास्तुकला में स्लिम और ऊर्ध्वाधर डिजाइन है। इसका अल्यूमीनियम वर्टिकल पिन और लो-ई डबल ग्लेज़िंग की विशेषताएं हैं। साथ ही, रूफ़टॉप सुविधाओं को छुपाने के लिए एक एकीकृत डिजाइन है।
होटल और कार्यालय सुविधाओं के साथ बिल्डिंग वेस्ट टावर है, और केवल कार्यालय सुविधाओं वाली बिल्डिंग ईस्ट टावर है। होटल का नाम 'जोसुन पेलेस सियोल गंगनाम' है और इसमें 254 कमरे हैं।
मैं आशा करता हूं कि यह जानकारी आपके लिए उपयोगी होगी। यदि आप और कुछ जानना चाहते हैं, तो कृपया पूछें।


In [59]:
polly_client = boto3.client('polly', region_name="ap-northeast-2")

In [94]:
response = polly_client.synthesize_speech(
    Text=script,
    OutputFormat="mp3",
    VoiceId="Seoyeon"
)

In [93]:
response['AudioStream'].read()

b'ID3\x04\x00\x00\x00\x00\x00#TSSE\x00\x00\x00\x0f\x00\x00\x03Lavf58.76.100\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xf3`\xc4\x00\x1e[\xfa\x00\x01O(\x01\x88\xa7%b\xdeB\xd0\xb8\xcf"o\xffM\x7fzR\x94\xa5)H\x81\xf0\xf8|\\\xe79\xc3\xe1\xc0\xe1\x10\x81\xc0\x108\xa0 \x10QNs\xbc\xf9*s\xce\xfdNw\xff\x9c\xe79\xces\x9c\x84!\x08B\x13\x9c\xef#)\x08F\xa1\x1b\xf9\xdfB\x10\x84%HJ\x9c\xe7?\xfes\xfa\x9c\xe4!\x08B\x10\x8dS\x9c\xefB\x1c\xe7?\xd4\xe79\xce\xf49\xce\x7f\xfc\xe79\xc8B~B\x13\xff\xff\xf2\x10\\\xe1\xe1\xea\xf7\xef<\xb2\x8e\x04\xb3\x83\x00\x9e\xa8]*\xddDYA?\xd20A\xff\xf3b\xc4\x15$\xd2R\x9d\x95\x89x\x00BI\xf5z\x99@\x06\x1d\x97\xc2a\x1e\x97 \x02\xbe-\x81\xd6\xd2\xac\x9e\x9a\xca\xf2\x8d\x08/\xabq\xd5\x8c\xf6\xa6\xe9\xa1n\x05\xa0t!hB\x18J\x04\xcd\xa5\xe7\xfa\xd7\xd6\x95*\xa7\xaa\xd5\xc2\xec\xf9[\xc3$t\xe2\xa1\xde\xe2D\xde\xb5\xff\x0e\xe3d\x9c\xa7\x8f\xf6\xfa?\xb2\x18\x86!p\x9eG\xa4H\xff{\xfb\xde\xaf\xbf\xbd\xfa\xc8\xfa\x0b\x02\xb9\x85\xa9\\\xdbhY\xa5$y3S\x04J@\x88\x04\xff8\x0e\x19w\xb8\x82*\xb

In [95]:
Audio(data=response['AudioStream'].read())

In [129]:
Audio('speech_20241104145708345.mp3')

#### 근처 맛집

In [230]:
map_area = driver.find_element(By.CSS_SELECTOR, 'g-img#lu_map').find_element(By.XPATH, '..')
map_src = map_area.get_attribute('data-url')

In [150]:
map_url = map_src.split('maps/')[0] + 'maps/search/음식점' + map_src.split('maps/')[1]

In [151]:
driver.get(map_url)

------

#### url로 검색

In [231]:
image_url = 'https://search.pstatic.net/sunny/?src=https%3A%2F%2Fp1.pxfuel.com%2Fpreview%2F32%2F834%2F438%2Fparis-tour-eifel-tower-tourism.jpg&type=sc960_832'

In [232]:
driver.get('https://images.google.com/')

In [233]:
driver.find_element(By.CSS_SELECTOR, "div[aria-label='이미지로 검색']").click()

In [234]:
url_input = driver.find_element(By.CSS_SELECTOR, "input[placeholder='이미지 링크 붙여넣기']")

In [235]:
url_input.send_keys(image_url)

In [236]:
driver.find_element(By.CSS_SELECTOR, "div.PXT6cd").find_element(By.CSS_SELECTOR, "div[role='button']").click()

In [237]:
try:
    google_search = driver.find_element(By.CSS_SELECTOR, "div.wNPKTe")
    google_search.click()
except:
    None

In [238]:
driver.switch_to.window(driver.window_handles[-1])

In [239]:
wiki = driver.find_element(By.CSS_SELECTOR, "div[data-attrid='description']")

In [240]:
wiki_url = wiki.find_element(By.CSS_SELECTOR, 'a[href]').get_attribute('href')

In [241]:
response = requests.get(wiki_url)

In [242]:
soup = BeautifulSoup(response.text)

In [243]:
content = soup.find('div', {"id":"mw-content-text"}).text.strip()

In [244]:
messages =  [{"role": "user", "content": content}]

In [245]:
body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 8192,
        "system": system_prompt,
        "messages": messages
    }  
)

In [246]:
response = bedrock.invoke_model(body=body, modelId=model_id)

In [247]:
response_body = json.loads(response.get('body').read())

In [248]:
script = response_body['content'][0]['text']

In [249]:
script = re.sub('\n+', '\n', script)

In [250]:
print(script)

에펠탑에 대한 주요 정보를 요약하면 다음과 같습니다:
- 프랑스 파리의 상징적인 철탑으로, 1889년 파리 만국박람회를 위해 건설되었습니다.
- 귀스타브 에펠이 설계했으며, 높이는 324m입니다. 
- 처음에는 20년 후 철거 예정이었지만, 무선 통신 안테나로 활용되면서 존속하게 되었습니다.
- 초기에는 '흉물'이라는 비난을 받았지만, 현재는 파리의 대표적인 랜드마크이자 관광명소가 되었습니다.
- 3층 구조로 되어 있으며, 1층과 2층에는 전망대와 레스토랑이 있습니다. 
- 7년마다 도색 작업을 하며, 야간 조명으로 아름다운 야경을 연출합니다.
- 매년 수백만 명의 관광객이 방문하는 세계적인 관광 명소입니다.
- 영화나 문화 콘텐츠에 자주 등장하며 파리와 프랑스를 상징하는 아이콘이 되었습니다.
에펠탑은 독특한 디자인과 역사적 의미, 그리고 파리의 상징성으로 인해 전 세계인의 사랑을 받는 건축물이 되었습니다.


In [185]:
Audio('speech_20241104153845182.mp3')

------

#### 번역 테스트

In [196]:
system_prompt = '''
[role]
- 당신은 주어진 장소의 정보를 읽고 그 장소를 여행하는 사람에게 정보를 정리하여 전달하는 아나운서입니다.
- 정보는 스페인어로 번역하여 전달합니다.
- 장소의 정보는 여행하는 사람이 흥미있어할 만한 내용을 위주로 정리합니다.
- 흥미있는 내용이 있다면 꼭 포함해주세요.

[current_year]
2024

[behavior_guidelines
- 정확한 정보만 제공하며, 문서에 없는 내용을 상상하여 답변하지 않습니다.
- '문서에에 따르면'과 같이 문서를 참고하는 것을 티내지 마세요.
- 사용자에게 바로 설명할 수 있도록 답변하세요.
- 아나운서의 대본처럼 바로 읽을 수 있게 답변하세요
'''

In [197]:
messages =  [{"role": "user", "content": content}]

In [198]:
body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 8192,
        "system": system_prompt,
        "messages": messages
    }  
)

In [199]:
response = bedrock.invoke_model(body=body, modelId=model_id)

In [200]:
response_body = json.loads(response.get('body').read())

In [201]:
script = response_body['content'][0]['text']

In [202]:
script = re.sub('\n+', '\n', script)

In [203]:
print(script)

Aquí está un resumen en español de la información sobre la Torre Eiffel:
"Estimados viajeros:
Hoy les hablaré sobre uno de los monumentos más emblemáticos de París y del mundo: la Torre Eiffel. 
Esta impresionante torre de hierro fue construida entre 1887 y 1889 para la Exposición Universal de París, que celebraba el centenario de la Revolución Francesa. Fue diseñada por el ingeniero Gustave Eiffel, de quien toma su nombre.
Con sus 324 metros de altura, la Torre Eiffel fue la estructura más alta del mundo hasta 1930. Está ubicada en el Campo de Marte, a orillas del río Sena. 
Algunos datos interesantes:
- Se utilizaron más de 18,000 piezas de hierro y 2.5 millones de remaches en su construcción.
- La torre pesa unas 10,000 toneladas en total.
- Se balancea unos 6-7 cm con el viento.
- Se repinta cada 7 años, utilizando 60 toneladas de pintura.
- Recibe alrededor de 7 millones de visitantes al año.
La Torre Eiffel tiene 3 niveles accesibles al público:
- El primer piso, a 57 metros de a

In [204]:
Audio('speech_20241105003059501.mp3')

In [27]:
with open('Team9-SampyeongLuckyStar 1293228564f18000a165e95f0d04de41.md', 'r') as f:
    md = f.read()

In [28]:
print(md)

# Team9-SampyeongLuckyStar

## 구현 애플리케이션 개요

### 프로젝트 이름

- 여행찬호

### 프로젝트 배경

- 여행지에는 다양한 언어와 문화의 관광객들이 찾는다. 유명한 관광지는 각 언어로 작성된 가이드가 있지만, 그렇지 않은 관광지도 많다.
- 생성형 AI를 활용해 랜드마크, 관광지 등의 정보를 사용자의 언어로 제공받고 주변 맛집 등의 정보를 제공받는다.

### 해결 아이디어

- 랜드마크나 맛집 이미지를 입력하여 정보를 검색한다.
- 검색된 관광지의 정보를 LLM에 전달하여 가이드를 생성한다.
- 생성된 가이드는 TTS를 이용하여 음성으로 변환한다.
- 검색된 관광지 주변의 음식점을 찾는다.
- 음식점의 리뷰를 LLM으로 분석하고, 경로를 제공한다.

### 액션아이템

- 웹 스크래핑
    - 이미지 검색 구현
        - [x]  이미지를 직접 올려서 구현 → Selenium
        - [x]  이미지 url을 올려서 구현
            - url 생성 : imgur, github, notion
            - 검색 → Selenium, 서드파티 api
    - 텍스트 검색
        - [ ]  메인 페이지 진입점 시나리오
    - 주변 맛집 검색
        - [ ]  검색된 장소 → 주변 맛집 찾는 시나리오(구글 지도)
            - url 패턴을 찾아서 url을 호출하여 스크래핑
            - 셀레니움으로 스크래핑
        - [ ]  맛집 검색 이후 시나리오
            - 리뷰 요약(LLM, Claude)
            - 길안내(url)
    - 장소 정보 제공 사이트
        - [ ]  위키백과
        - [ ]  나무위키
        - [ ]  트립어드바이저 등 제 3의 사이트
- 음식점, 미술품으로 확장 가능한지 Test
    - [ ]  음식점 간판
    - [ ]  미술품(조각상, 그림)
-

In [29]:
system_prompt = '''
주어진 마크다운 문서를 읽고 github README.md를 작성해주세요
'''

In [30]:
messages =  [{"role": "user", "content": md}]

In [31]:
body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 8192,
        "system": system_prompt,
        "messages": messages
    }  
)

In [32]:
response = bedrock.invoke_model(body=body, modelId=model_id)

In [33]:
response_body = json.loads(response.get('body').read())

In [34]:
print(response_body['content'][0]['text'])

# Team9-SampyeongLuckyStar

## 여행찬호 - AI 기반 여행 가이드 애플리케이션

### 프로젝트 개요

여행찬호는 생성형 AI를 활용하여 다국어 여행 가이드 서비스를 제공하는 애플리케이션입니다. 사용자가 랜드마크나 관광지의 이미지를 입력하면, AI가 해당 장소에 대한 정보를 분석하고 사용자의 언어로 가이드를 생성합니다. 또한 주변 맛집 정보와 경로 안내 기능을 제공하여 여행자들에게 편리한 경험을 선사합니다.

### 주요 기능

1. 이미지 기반 장소 인식
2. AI 생성 다국어 가이드
3. 음성 변환 가이드 (TTS)
4. 주변 맛집 검색 및 리뷰 분석
5. 경로 안내

### 기술 스택

- 웹 스크래핑: Selenium, BeautifulSoup
- AI/ML: Amazon Bedrock, LLM
- 프론트엔드: Streamlit
- 기타: Google Maps API

### 개발 현황

- [x] 이미지 검색 구현 (직접 업로드 및 URL 입력)
- [ ] 텍스트 기반 검색
- [ ] 주변 맛집 검색 및 리뷰 분석
- [ ] 장소 정보 제공 사이트 스크래핑 (위키백과, 나무위키, 트립어드바이저)
- [ ] Streamlit을 이용한 웹 인터페이스 구현
- [ ] 음식점 간판 및 미술품 인식 확장

### 향후 계획

- 사용자 플로우 및 시나리오 개선
- 다양한 언어 지원 확대
- 모바일 앱 버전 개발
- 사용자 피드백 기반 기능 개선

### 팀원

- 스테디: 사용자 플로우 및 시나리오 개발
- 단톤: 웹 스크래핑 POC
- 제인: 장소 정보 제공 사이트 스크래핑
- 이든: 서비스 진입점 시나리오 및 시스템 프롬프트 POC
- 투나: Streamlit 페이지 구현 및 백업 시나리오 개발

### 참고 자료

- [Amazon Bedrock 튜토리얼](https://www.notion.so/Amazon-Bedrock-1193228564f1804b8c5af2c60d79f35c?pvs=21)
- [해커톤 오리엔테이션 영상](ht